In [239]:
import requests
import lxml.html
import bs4

import numpy as np
import pandas as pd

In [17]:
USER_AGENT = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
LOGIN_URL = 'https://www.placement.iitbhu.ac.in/accounts/login'
COMPANY_URL = 'https://www.placement.iitbhu.ac.in/company/calendar'
USERNAME = 'sartaj.singh.apm13@itbhu.ac.in'
PASSWORD = 'Superkid@123'

In [ ]:
data_mapper = {
    'updated_at': 'updated_at',
    'company_name': 'company_name',
    'profile': 'company_profile',
    'purpose': 'purpose',
    'x_percent': 'x',
    'xii_percent': 'xii',
    'cgpa': 'cgpa',
    'course': 'course',
    'dept': 'dept',
    'active_backlog': 'a_backlog',
    'total_backlog': 't_backlog',
    'ppt_date': 'ppt_date',
    'exam_date': 'exam_date',
    'interview_date': 'interview_date',
    'status': 'status',
    'branch_issue_deadline': 'branch_issue_dead',
    'willingness_deadline': 'willingness_dead'    
}

In [18]:
def make_login_form(session, username, password):
    login = session.get(LOGIN_URL)
    login_html = lxml.html.fromstring(login.text)
    hidden_inputs = login_html.xpath(r'//form//input[@type="hidden"]')
    form = {x.attrib['name']: x.attrib['value'] for x in hidden_inputs}
    form['login'] = username
    form['password'] = password
    return form

In [19]:
session = requests.session()

In [20]:
form = make_login_form(session, USERNAME, PASSWORD)
form

{'csrfmiddlewaretoken': '4DXbPQtbBAGO8kGzzzipS41WSmn0BTVb',
 'login': 'sartaj.singh.apm13@itbhu.ac.in',
 'password': 'Superkid@123'}

In [42]:
params = {'Referer': LOGIN_URL, 'User-Agent': USER_AGENT}
login = s.post(LOGIN_URL, data=form, headers=params)
login

<Response [200]>

In [141]:
params = {'page_size': 10}
data = s.get(COMPANY_URL, params=params)
data

UnboundLocalError: local variable 'bypass' referenced before assignment

In [ ]:
company_data = bs4.BeautifulSoup(data.text, 'html.parser')

In [165]:
def get_data(data_mapper):
    companies = []
    for company_div in company_data.find_all('div', attrs={'class': 'row company'}):
        company = {}
        for name, class_name in data_mapper.items():
            company[name] = company_div.find(attrs={'class': class_name}).text
        companies.append(company)
    return companies

In [217]:
clips = {
    'active_backlog': 'Active backlog allowed :'.find(':') ,
    'branch_issue_deadline': 'Branch Issues Deadline :'.find(':') + 1,
    'cgpa': 'cgpa :'.find(':'),
    'course': 'Course(s) :'.find(':'),
    'dept': 'Department(s) :'.find(':'),
    'exam_date': 'Exam Date :'.find(':'),
    'interview_date': 'Interview Date :'.find(':') ,
    'ppt_date': 'PPT Date : '.find(':'),
    'status': 'Status : '.find(':'),
    'total_backlog': 'Total backlog allowed :'.find(':'),  
    'updated_at': 'st Updated:'.find(':') + 2,
    'willingness_deadline': 'Willingness Deadline :'.find(':') + 1,
    'x_percent': 'X :'.find(':'),
    'xii_percent': 'XII :'.find(':')
}

In [231]:
def clip_data(companies, clips):
    for company in companies:
        for clip_name, clip_num in clips.items():
            num = clip_num + 2
            company[clip_name] = company[clip_name][num:]
    return companies

In [299]:
def process_df(companies):
    df = pd.DataFrame(companies)
    df.loc[:, 'active_backlog'] = df['active_backlog'].replace({'N/A': np.nan}).astype(np.float64)
    df.loc[:, 'total_backlog'] = df['total_backlog'].replace({'N/A': np.nan}).astype(np.float64)
    df.loc[:, 'cgpa'] = df['cgpa'].astype(np.float64)
    df.loc[:, 'exam_date'] = pd.to_datetime(df['exam_date'])
    df.loc[:, 'interview_date'] = pd.to_datetime(df['interview_date'])
    df.loc[:, 'ppt_date'] = pd.to_datetime(df['ppt_date'])
    #df.loc[:, 'branch_issue_deadline'] = pd.to_datetime(df['branch_issue_deadline']) unknown
    df.loc[:, 'updated_at'] = pd.to_datetime(df['updated_at'])
    df.loc[:, 'x_percent'] = df['x_percent'].str[:-1].astype(np.float64)
    df.loc[:, 'xii_percent'] = df['xii_percent'].str[:-1].astype(np.float64)
    df.loc[:, 'profile'] = df['profile'].replace({'': np.nan})
    df.loc[:, 'willingness_deadline'] = df['willingness_deadline'].replace({'': np.nan})
    return df

In [300]:
companies = get_data(data_mapper)
clip_data(companies, clips)
df = process_df(companies)
df

,active_backlog,branch_issue_deadline,cgpa,company_name,course,dept,exam_date,interview_date,ppt_date,profile,purpose,status,total_backlog,updated_at,willingness_deadline,x_percent,xii_percent
0,NaN,"Wed, 20 Sep 2017 | 10 AM",7.0,Deutsche Bank,mtech,ece,NaT,NaT,NaT,Analyst,placement 2017-18,Open for Willingness,NaN,2017-09-19 16:52:00,"Wed, 20 Sep 2017 | 10 AM",70.0,70.0
1,NaN,"Wed, 20 Sep 2017 | 10 AM",0.0,Amazon,btechidd,cse eee ece mat,NaT,NaT,NaT,NaN,placement 2017-18,Under Branch Issue,NaN,2017-09-19 16:14:00,NaN,0.0,0.0
2,NaN,"Wed, 6 Sep 2017 | 12 PM",6.0,ACCOLITE,btechidd,cse eee ece mat,NaT,NaT,NaT,Software Engineer,placement 2017-18,Open for Willingness,NaN,2017-09-19 15:53:00,"Wed, 20 Sep 2017 | 3 PM",0.0,0.0
3,NaN,"Wed, 20 Sep 2017 | 10 AM",6.0,I Cube Nanotec,btech,cer,NaT,NaT,NaT,NaN,placement 2017-18,Under Branch Issue,NaN,2017-09-19 14:15:00,NaN,60.0,60.0
4,NaN,"Wed, 20 Sep 2017 | 10 AM",6.5,Tata Steel,btech,cer eee,NaT,NaT,NaT,Management Trainee Technical,placement 2017-18,Under Branch Issue,NaN,2017-09-19 14:05:00,NaN,60.0,60.0
5,NaN,"Wed, 20 Sep 2017 | 10 AM",0.0,CAPGEMINI,btechidd,cer che civ cse eee ece mec met min bce bme ms...,NaT,NaT,NaT,NaN,placement 2017-18,Under Branch Issue,NaN,2017-09-19 13:37:00,NaN,0.0,0.0
6,NaN,"Wed, 20 Sep 2017 | 10 AM",7.0,Orbees,idd,civ eee mec phe bce bme,NaT,NaT,NaT,Associate Consultant,placement 2017-18,Under Branch Issue,NaN,2017-09-19 13:27:00,NaN,0.0,0.0
7,0.0,"Wed, 12 Jul 2017 | 11 PM",0.0,Futures First,btechidd,cer che civ cse eee ece mec met min phe bce bm...,2017-09-23,NaT,2017-09-23,Trainee-Market Analyst,placement 2017-18,Open for Willingness,NaN,2017-09-19 12:28:00,"Wed, 20 Sep 2017 | 10 AM",0.0,0.0
8,NaN,"Thu, 21 Sep 2017 | 10 AM",0.0,Amrita University,phd,civ cse eee ece mec,NaT,NaT,NaT,Assistant Professor,placement 2017-18,Under Branch Issue,NaN,2017-09-19 01:09:00,NaN,0.0,0.0
9,NaN,"Thu, 21 Sep 2017 | 10 AM",6.0,Mercedes Benz,iddmtech,eee ece mat,NaT,NaT,NaT,PGET,placement 2017-18,Under Branch Issue,NaN,2017-09-19 01:01:00,NaN,60.0,60.0


In [301]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 17 columns):
active_backlog           1 non-null float64
branch_issue_deadline    10 non-null object
cgpa                     10 non-null float64
company_name             10 non-null object
course                   10 non-null object
dept                     10 non-null object
exam_date                1 non-null datetime64[ns]
interview_date           0 non-null datetime64[ns]
ppt_date                 1 non-null datetime64[ns]
profile                  7 non-null object
purpose                  10 non-null object
status                   10 non-null object
total_backlog            0 non-null float64
updated_at               10 non-null datetime64[ns]
willingness_deadline     3 non-null object
x_percent                10 non-null float64
xii_percent              10 non-null float64
dtypes: datetime64[ns](4), float64(5), object(8)
memory usage: 1.4+ KB


In [302]:
df.to_csv('companies_10.csv', index=False)
pd.read_csv('companies_10.csv')

,active_backlog,branch_issue_deadline,cgpa,company_name,course,dept,exam_date,interview_date,ppt_date,profile,purpose,status,total_backlog,updated_at,willingness_deadline,x_percent,xii_percent
0,NaN,"Wed, 20 Sep 2017 | 10 AM",7.0,Deutsche Bank,mtech,ece,NaN,NaN,NaN,Analyst,placement 2017-18,Open for Willingness,NaN,2017-09-19 16:52:00,"Wed, 20 Sep 2017 | 10 AM",70.0,70.0
1,NaN,"Wed, 20 Sep 2017 | 10 AM",0.0,Amazon,btechidd,cse eee ece mat,NaN,NaN,NaN,NaN,placement 2017-18,Under Branch Issue,NaN,2017-09-19 16:14:00,NaN,0.0,0.0
2,NaN,"Wed, 6 Sep 2017 | 12 PM",6.0,ACCOLITE,btechidd,cse eee ece mat,NaN,NaN,NaN,Software Engineer,placement 2017-18,Open for Willingness,NaN,2017-09-19 15:53:00,"Wed, 20 Sep 2017 | 3 PM",0.0,0.0
3,NaN,"Wed, 20 Sep 2017 | 10 AM",6.0,I Cube Nanotec,btech,cer,NaN,NaN,NaN,NaN,placement 2017-18,Under Branch Issue,NaN,2017-09-19 14:15:00,NaN,60.0,60.0
4,NaN,"Wed, 20 Sep 2017 | 10 AM",6.5,Tata Steel,btech,cer eee,NaN,NaN,NaN,Management Trainee Technical,placement 2017-18,Under Branch Issue,NaN,2017-09-19 14:05:00,NaN,60.0,60.0
5,NaN,"Wed, 20 Sep 2017 | 10 AM",0.0,CAPGEMINI,btechidd,cer che civ cse eee ece mec met min bce bme ms...,NaN,NaN,NaN,NaN,placement 2017-18,Under Branch Issue,NaN,2017-09-19 13:37:00,NaN,0.0,0.0
6,NaN,"Wed, 20 Sep 2017 | 10 AM",7.0,Orbees,idd,civ eee mec phe bce bme,NaN,NaN,NaN,Associate Consultant,placement 2017-18,Under Branch Issue,NaN,2017-09-19 13:27:00,NaN,0.0,0.0
7,0.0,"Wed, 12 Jul 2017 | 11 PM",0.0,Futures First,btechidd,cer che civ cse eee ece mec met min phe bce bm...,2017-09-23 00:00:00,NaN,2017-09-23 00:00:00,Trainee-Market Analyst,placement 2017-18,Open for Willingness,NaN,2017-09-19 12:28:00,"Wed, 20 Sep 2017 | 10 AM",0.0,0.0
8,NaN,"Thu, 21 Sep 2017 | 10 AM",0.0,Amrita University,phd,civ cse eee ece mec,NaN,NaN,NaN,Assistant Professor,placement 2017-18,Under Branch Issue,NaN,2017-09-19 01:09:00,NaN,0.0,0.0
9,NaN,"Thu, 21 Sep 2017 | 10 AM",6.0,Mercedes Benz,iddmtech,eee ece mat,NaN,NaN,NaN,PGET,placement 2017-18,Under Branch Issue,NaN,2017-09-19 01:01:00,NaN,60.0,60.0
